In [14]:

import cv2
import numpy as np
import tensorflow as tf

model_paths = [
    "P3_Deteccion_numeros_1.h5",
    "P3_Deteccion_numeros_2.h5",
    "P3_Deteccion_numeros_3.h5",
    "P3_Deteccion_numeros_4.h5",
    "P3_Deteccion_numeros_6.h5",
    "P3_Deteccion_numeros_Final.h5"    
]

models = [tf.keras.models.load_model(path) for path in model_paths]

cap = cv2.VideoCapture(0)

ROI_SIZE = 200  

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        h, w, _ = frame.shape

        cx, cy = w // 2, h // 2

        x1 = cx - ROI_SIZE // 2
        y1 = cy - ROI_SIZE // 2
        x2 = cx + ROI_SIZE // 2
        y2 = cy + ROI_SIZE // 2

        x1 = max(0, x1); y1 = max(0, y1)
        x2 = min(w, x2); y2 = min(h, y2)
        
        roi = frame[y1:y2, x1:x2]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        _, threshed = cv2.threshold(gray, 155, 255, cv2.THRESH_BINARY_INV)
        resized = cv2.resize(threshed, (28, 28), interpolation=cv2.INTER_AREA)
        normalized = resized.astype("float32") / 255.0
        inverted = 1.0 - normalized
        input_img = inverted.reshape(1, 28, 28, 1)

        predictions = []  

        for m in models:
            probs = m.predict(input_img, verbose=0)[0]  
            pred_label = int(np.argmax(probs))
            prob_max = float(np.max(probs))
            predictions.append((pred_label, prob_max))

        base_x = 10
        base_y = 30
        dy = 25

        for i in range(5):
            pred, prob = predictions[i]
            text = f"M{i+1}: {pred}  Prob: {prob:.3f}"
            cv2.putText(
                frame,
                text,
                (base_x, base_y + i * dy),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 0, 255),   # ROJO
                2,
                cv2.LINE_AA
            )

        # Modelo final (6) en verde
        final_pred, final_prob = predictions[5]
        text_final = f"Modelo Final: {final_pred}  Prob: {final_prob:.3f}"
        cv2.putText(
            frame,
            text_final,
            (base_x, base_y + 6 * dy),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),     # VERDE
            2,
            cv2.LINE_AA
        )

        # Mostrar frame con predicciones
        cv2.imshow('Real-time prediction (ROI centrado, sin cuadro)', frame)

        # Mostrar solo el ROI procesado, escalado para verlo grande (zoom)
        upscaled_resized = cv2.resize(resized, (280, 280), interpolation=cv2.INTER_NEAREST)
        cv2.imshow('ROI binarizado 28x28 (escalado)', upscaled_resized)

        # Salir con 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    pass
finally:
    cap.release()
    cv2.destroyAllWindows()
